In [1]:
import trackintel as ti
import pandas as pd

In [2]:
# Helper function to generate subsequences of length k
def generate_subsequences(sequences):
    subsequences = []
    for seq in sequences:
        for seq_2 in sequences:
            if seq != seq_2:
                if len(seq) == 1 and len(seq[0])==1: #length 1 data
                    elements_append = [[seq[0], seq[0]],[seq_2[0], seq_2[0]],[seq[0], seq_2[0]],
                                       [seq_2[0], seq[0]],[{list(seq[0])[0], list(seq_2[0])[0]}]]
                    
                    for element in elements_append:
                        if element not in subsequences:
                            subsequences.append(element)
                elif merge_validity(seq, seq_2): #length more than 1
                    if len(seq_2[-1])==1:
                        if (seq+[seq_2[-1]]) not in subsequences:
                            subsequences.append(seq+[seq_2[-1]])
                    else:
                        result=[]
                        for sets in seq:
                            result.append(sets.copy())
                        for num in seq_2[-1]:
                            if num not in result[-1]:
                                result[-1].add(num)
                                break
                        if result not in subsequences:
                            subsequences.append(result)
    return subsequences

# Helper function to calculate support of a sequence in the dataset
def calculate_support(sequences, subsequences):
    support_subsequences, support_count = [], []
    for subseq in subsequences:
        for seq in sequences:
            if is_subsequence(seq, subseq):
                if subseq not in support_subsequences:
                    support_subsequences.append(subseq)
                    support_count.append(1)
                else:
                    support_count[support_subsequences.index(subseq)] += 1
    return support_subsequences, support_count

#helper function to check for merge validity
def merge_validity(seq1, seq2):
    sequence1 = seq1.copy()
    sequence2 = seq2.copy()  
    
    if len(sequence1[0])==1: #seq 1 first set only has 1 element
        sequence1 = sequence1[1:]
    elif sequence2[0].issubset(sequence1[0]) and len(sequence1[0])==len(sequence2[0])+1:
        sequence1[0] = sequence2[0]
    elif len(sequence1)==1 and len(sequence2)==1: #both sets have 1 sequence only
        for num in sequence1[0]:
            for num2 in sequence2[0]:
                set1 = sequence1[0].copy()
                set2 = sequence2[0].copy()
                set1.remove(num)
                set2.remove(num2)
                if set1 == set2:
                    return True
        return False
    else:
        return False
    
    if len(sequence2[-1])==1: #seq 1 last set only has 1 element
        sequence2 = sequence2[:len(sequence2)-1]
    elif sequence1[-1].issubset(sequence2[-1]) and len(sequence2[-1])==len(sequence1[-1])+1:
        sequence2[-1] = sequence1[-1]
    else:
        return False
    
    return (sequence1==sequence2)

# Helper function to check if subseq is a subsequence of seq
def is_subsequence(sequence, subseq):
    if len(subseq)>len(sequence):
        return False
    
    index=0 #index for subsequence
    #iterate in order of sequence
    for set in sequence:
        if subseq[index].issubset(set):
            index+=1
        
        #stop when subsequence is verified
        if index == len(subseq):
            break
    
    #when all subseq can be found in 1 of the sequences
    if index==len(subseq):
        return True
    return False

# Generalized Sequential Pattern (GSP) Algorithm
def GSP(sequences, min_support):
    all_patterns, all_sup = [], []
    subsequences = []
    print("Obtaining 1-sequence itemset")
    for user in sequences:
        for set in user:
            set_list = list(set)
            for coordinates in set_list:
                if [{coordinates}] not in subsequences:
                    subsequences.append([{coordinates}])
    print(f"Length of 1 sequence itemset: {len(subsequences)}")
    print()

    k=1
    while subsequences:
        print(f"Finding frequent subsequences of length {k}...")
        support_subsequences, support_count = calculate_support(sequences, subsequences)

        frequent_subsequences, frequent_sup =[], []
        for subseq in support_subsequences:
            if support_count[support_subsequences.index(subseq)]>=min_support:
                frequent_subsequences.append(subseq)
                frequent_sup.append(support_count[support_subsequences.index(subseq)])
        all_patterns.extend(frequent_subsequences)
        all_sup.extend(frequent_sup)
        print(f"Length of {k} sequence frequent itemset: {len(frequent_subsequences)}")
        print()
        
        k+=1
        print(f"Generating candidates for length {k}")
        subsequences = generate_subsequences(frequent_subsequences)
        print(f"Length of {k} sequence itemset: {len(subsequences)}")
        print()
    
    return all_patterns, all_sup

In [3]:
cityB = ti.io.read_triplegs_csv("city_B_triplegs.csv", index_col=None)

In [ ]:
frequent_list,cityB_set = [],[]

for i in cityB['user_id'].unique():
    cityB_geom = list(cityB[cityB['user_id']==i]['geom'])
    cityB_user = []
    for geom in cityB_geom:
        geom_set=set()
        for coordinates in geom.coords:
            geom_set.add(coordinates)
        cityB_user.append(geom_set)
    cityB_set.append(cityB_user)

minsup = int(cityB['user_id'].unique().size/100)
print(f"minsup: {minsup}")

# Run the GSP algorithm
frequent_patterns, support_count = GSP(cityB_set, minsup)

# Output the frequent sequential patterns
cityB_frequent_df = pd.DataFrame(list(zip(frequent_patterns, support_count)), columns=['Subsequence', 'Support'])
cityB_frequent_df.head()

minsup: 248
Obtaining 1-sequence itemset
Length of 1 sequence itemset: 14319

Finding frequent subsequences of length 1...
Length of 1 sequence frequent itemset: 420

Generating candidates for length 2
Length of 2 sequence itemset: 264390

Finding frequent subsequences of length 2...
Length of 2 sequence frequent itemset: 279

Generating candidates for length 3
Length of 3 sequence itemset: 3150

Finding frequent subsequences of length 3...
Length of 3 sequence frequent itemset: 144

Generating candidates for length 4
Length of 4 sequence itemset: 637

Finding frequent subsequences of length 4...
Length of 4 sequence frequent itemset: 88

Generating candidates for length 5
Length of 5 sequence itemset: 154

Finding frequent subsequences of length 5...
Length of 5 sequence frequent itemset: 1

Generating candidates for length 6
Length of 6 sequence itemset: 0



,Subsequence,Support
0,"[{(101.0, 80.0)}]",1094
1,"[{(100.0, 80.0)}]",1148
2,"[{(101.0, 82.0)}]",714
3,"[{(99.0, 80.0)}]",392
4,"[{(102.0, 83.0)}]",307


In [5]:
cityB_frequent_df.to_csv("cityB_task2_result.csv", index=False)